In [1]:
library(ggplot2)
library(car)
options(width=160, repr.plot.width=11, repr.plot.height=7) 

ERROR: Error in library(car): there is no package called ‘car’


# Úkoly

V rámci cvičení provedeme analýzu datasetu o tučňácích z knihovny `palmerpenguins`

https://cran.r-project.org/web/packages/palmerpenguins/index.html

Budeme zkoumat váhu tučňáků v závislosti na různých kategoriálních vysvětlujících proměnných.

## Úkol 1.a: Příprava dat

Nejprve je třeba načíst a připravit data.

**Data pro analýzu:**
- knihovna: `palmerpenguins`
- dataset `penguins`

**Sledované proměnné**
- `body_mass_g` - váha tučňáka v gramech - zkoumaná proměnná. 
- `species` - druh tučňáka, kategorická proměnná s úrovněmi Adelie, Chinstrap a Gentoo  
- `sex` - pohlaví tučňáka - kategorická proměnná s úrovněmi male, female, NA 
- `island` - ostrov, kde se tučňák vyskytuje - kategorická proměnná s úrovněmi Biscoe, Dream a Torgersen

Dataset upravte, aby neobsahoval řádky s tučňáky s neznámým pohlavím.

In [ ]:
#install.packages("palmerpenguins")
library(palmerpenguins)
dat=subset(penguins,!is.na(sex))

## Úkol 1.b: Průzkum dat

Prozkoumejte jednotlivé proměnné - u vysvětlované proměnné najděte základní popisné statistiky, u faktorových zjistěte, kolik je záznamů v každé kategoriemi.

In [ ]:
summary(dat)

## Úkol 1.c: Závislost váhy na druhu - základní přehled

Prozkoumejte číselně i graficky váhu tučňáků zvlášť podle druhu. Zkuste odhadnout, jestli bude napříč druhyislB významný rozdíl.

In [ ]:
by(dat$body_mass_g,dat$species,summary)

In [ ]:
boxplot(body_mass_g~species,data=dat,col=c("magenta","cyan","darkblue"))

## Úkol 1.d: Závislost váhy na druhu - ověření předpokladů

Otestujte předpoklady modelu analýzy rozptylu jednoduchého třídění váhy podle druhu. Prozkoumejte normalitu i shodu rozptylů.

In [ ]:
r=lm(body_mass_g~species,data=dat)$resid
shapiro.test(r)
bartlett.test(body_mass_g~species,data=dat)
library(car)
leveneTest(body_mass_g~species,data=dat)

## Úkol 1.e: Závislost váhy na druhu - test shody středních hodnot

Aplikujte model analýzy rozptylu. Otestujte, zda je střední váha tučňáků stejná napříč jednotlivými druhy, nebo zda se významně liší. Výsledky testu interpretujte.
Příslušné součty čtverců a testovou statistiku zkuste spočítat jednak vestavěnou funkcí a jednak ručně.

In [ ]:
anova(aov(body_mass_g~species,data=dat))

In [ ]:
y=dat$body_mass_g
y1=subset(y,dat$species=="Adelie")
y2=subset(y,dat$species=="Chinstrap")
y3=subset(y,dat$species=="Gentoo")
St=sum((y-mean(y))^2)
St
Se=sum((y1-mean(y1))^2)+sum((y2-mean(y2))^2)+sum((y3-mean(y3))^2)
Se
St-Se
Sa=length(y1)*(mean(y1)-mean(y))^2+length(y2)*(mean(y2)-mean(y))^2+length(y3)*(mean(y3)-mean(y))^2
Sa

<font color="blue">**Nyní zpět k výkladu.**</font>

---

## Úkol 2.a: Závislost váhy na druhu - střední hodnoty

Najděte odhady středních hodnot pro každou z kategorií zvlášť.

In [ ]:
lm(body_mass_g~species,data=dat)
beta=lm(body_mass_g~species,data=dat)$coef
beta
print(c(beta[1],beta[1]+beta[2],beta[1]+beta[3]))

In [ ]:
by(dat$body_mass_g,dat$species,mean)

## Úkol 2.b: Závislost váhy na druhu - pořadí úrovní

Vyzkoušejte totéž, když se změní pořadí úrovní faktorové proměnné pomocí funkce `relevel()`.

In [ ]:
specrelevel=relevel(dat$species, ref = "Gentoo")
summary(lm(body_mass_g~specrelevel,data=dat))

## Úkol 2.c: Závislost váhy na druhu - mnohonásobné porovnání

Najděte, o kolik se jednotlivé odhady liší a testujte pomocí Tukeyho HSD testu, zda je nějaký z rozdílů významně velký.

In [ ]:
TukeyHSD(aov(body_mass_g~species,data=dat))

## Úkol 2.d: Práce s kontrasty

Najděte designovou matici $\mathbf X$ pro přislušný lineární model. Zkuste kontrasty reparametrizovat použitím modelu s jinou volbou `contr`.

In [ ]:
model.matrix(lm(body_mass_g~species,data=dat))

In [ ]:
fit2=lm(body_mass_g~species,data=dat,contr=list(species = contr.SAS))
summary(fit2)
model.matrix(fit2)

<font color="blue">**Nyní zpět k výkladu.**</font>

---

## Úkol 3.a: Dvojné třídění

Kromě závislosti váhy na druhu uvažujte i závislost na pohlaví. Odhadněte její vliv na střední hodnotu váhy a otestujte, zda je pro model významná.

In [ ]:
anova(aov(body_mass_g~species+sex,data=dat))

In [ ]:
summary(lm(body_mass_g~species+sex,data=dat))

## Úkol 3.b: Dvojné třídění s interakcemi

Přidejte do modelu i interakce druhu a pohlaví a zkoumejte jejich vliv.

In [ ]:
anova(aov(body_mass_g~species*sex,data=dat))

In [ ]:
summary(lm(body_mass_g~species*sex,data=dat))

## Úkol 3.c: Trojné třídění

Uvažujte navíc i ze kterého ostrova tučňáci pochází. Vyzkoušejte různé varianty modelů s interakcemi a bez nich. Interpretujte výsledky testů a hodnoty regresních koeficientů.

Hint: Ukáže se, že v některých kombinacích kategorií nejsou žádná pozorování. To můžeme obejít vhodným sloučením kategorií do jedné.

In [ ]:
anova(aov(body_mass_g~species+sex+island,data=dat))
summary(lm(body_mass_g~species+sex+island,data=dat))

In [ ]:
summary(lm(body_mass_g~species*sex*island,data=dat))
## nefunguje, protoze nejsou vsechny druhy na vsech ostrovech

In [ ]:
## podivame se na pocty v kategoriich
table(dat$species,dat$island)

---

---

In [ ]:
## sloucime promenne
species2=ifelse(dat$species=="Adelie","Adelie","Other")
island2=ifelse(dat$island=="Biscoe","Biscoe","Other")
table(species2,island2)

In [ ]:
summary(lm(body_mass_g~species2*sex*island2,data=dat))